In [1]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from torch.autograd import Variable
import sys
import random
from torch import nn
from torchvision import models
from torchvision.models import ResNeXt50_32X4D_Weights
import glob

In [50]:
path_to_videos = ['C:/Users/alapa/Documents/Deepfake_detection_using_deep_learning-master/Deepfake_detection_using_deep_learning-master/Celeb-DF/Celeb-real/id3_0008.mp4']
#path_to_videos = ['C:/Users/alapa/Downloads/fake.mp4']

frame_count = []
for video_file in path_to_videos:
  cap = cv2.VideoCapture(video_file)
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)

frames are  [458]


In [51]:
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(x_lstm[:,-1,:]))

In [52]:
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
sm = nn.Softmax(dim=1)
inv_normalize =  transforms.Normalize(mean=-1*np.divide(mean,std),std=np.divide([1,1,1],std))
def im_convert(tensor):
    """ Display a tensor as an image. """
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze()
    image = inv_normalize(image)
    image = image.numpy()
    image = image.transpose(1,2,0)
    image = image.clip(0, 1)
    cv2.imwrite('./2.png',image*255)
    return image

In [53]:

# Initialize lists to store predictions and confidence values for each frame
predictions = []
confidences = []

In [54]:
def predict(model, img, path='./'):
    # Ensure the model is in evaluation mode and the input is on the correct device
    model.eval()

    if not isinstance(img, torch.Tensor) or img.shape[0] == 0:
        print("Warning: Empty or invalid input image tensor.")
        return None

    # Ensure the input is on the correct device
    img = img.to('cuda')

    # Loop through each frame in the batch
    for i in range(img.size(1)):  # img.size(1) gives the number of frames
        frame = img[:, i, :, :, :].unsqueeze(0)  # Extract each frame and add batch dimension

        # Forward pass
        fmap, logits = model(frame)

        # Extract weights from the final linear layer
        try:
            weight_softmax = model.linear1.weight.detach().cpu().numpy()
        except AttributeError:
            print("Error: Model does not have a layer named 'linear1'.")
            return None

        # Softmax and prediction
        logits = torch.nn.functional.softmax(logits, dim=1)  # Ensure softmax is applied
        _, prediction = torch.max(logits, 1)
        confidence = logits[:, int(prediction.item())].item() * 100

        # Store results
        if int(prediction.item()) == 1:
            predictions.append(1)
        else:
            predictions.append(0)
        confidences.append(confidence)

        print(f'Frame {i + 1}: Prediction = {prediction.item()}, Confidence = {confidence:.2f}%')

    # Optionally, return the list of predictions and confidences
    return [predictions, confidences]



In [55]:
#new implementation

class validation_dataset(Dataset):
    def __init__(self,video_names,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self, idx):
        video_path = self.video_names[idx]
        frames = []

        # Extract all frames from the video
        for frame in self.frame_extract(video_path):
            frames.append(self.transform(frame))
        #print("frames: ",frames,"\n Length of frames: ",len(frames))
        # Stack all frames into a tensor
        frames = torch.stack(frames)
        #print("frames: ",frames,"\n Length of frames: ",len(frames))
        # Add an extra dimension (batch dimension)
        #print("frames: ",frames,"\n Length of frames: ",len(frames))
        return frames.unsqueeze(0)


    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path) 
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image
              
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1, 2, 0)
    b, g, r = cv2.split(image)
    image = cv2.merge((r, g, b))
    image = image * [0.22803, 0.22145, 0.216989] + [0.43216, 0.394666, 0.37645]
    image = image * 255.0
    plt.imshow(image.astype(int))
    plt.show()

In [56]:
train_transforms = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize((im_size,im_size)),
  transforms.ToTensor(),
  transforms.Normalize(mean,std)])

In [57]:

video_dataset = validation_dataset(path_to_videos,sequence_length = 20,transform = train_transforms)
model = Model(2).cuda()
path_to_model = 'C:/Users/alapa/Documents/Deepfake_detection_using_deep_learning-master/Deepfake_detection_using_deep_learning-master/train_model_97.49_epoch_40.pt'
model.load_state_dict(torch.load(path_to_model))
# Ensure the model is in evaluation mode
model.eval()
print("hi")

hi


In [58]:
print(len(predictions))
print(len(confidences))
for i in range(0,len(predictions)):
    print(predictions[i], confidences[i] )

0
0


In [59]:

# Initialize sum to accumulate confidence values
sum_confidence = 0

# Process the single video
p = predict(model, video_dataset[0], './')  # Assuming there's only one video in video_dataset

if p is not None:
    predictions = p[0]
    confidences = p[1]

    # Accumulate confidence values based on predictions
    for i in range(len(predictions)):
        if predictions[i] == 1:  # Assuming 1 means 'real'
            sum_confidence += confidences[i]
        else:  # Assuming 0 means 'fake'
            sum_confidence += 100 - confidences[i]

    # Calculate the average confidence
    avg_confidence = sum_confidence / len(predictions)

    # Determine the final prediction for the video
    if avg_confidence < 50:
        fake_confidence = 100 - avg_confidence
        print(f"Fake\nConfidence: {fake_confidence:.2f}%")
    else:
        print(f"Real\nConfidence: {avg_confidence:.2f}%")
else:
    print("Prediction failed for the video.")



Frame 1: Prediction = 1, Confidence = 60.38%
Frame 2: Prediction = 1, Confidence = 55.38%
Frame 3: Prediction = 1, Confidence = 54.09%
Frame 4: Prediction = 1, Confidence = 56.75%
Frame 5: Prediction = 1, Confidence = 55.88%
Frame 6: Prediction = 1, Confidence = 56.78%
Frame 7: Prediction = 1, Confidence = 55.99%
Frame 8: Prediction = 1, Confidence = 56.01%
Frame 9: Prediction = 1, Confidence = 55.64%
Frame 10: Prediction = 1, Confidence = 55.50%
Frame 11: Prediction = 1, Confidence = 58.01%
Frame 12: Prediction = 1, Confidence = 57.95%
Frame 13: Prediction = 1, Confidence = 59.63%
Frame 14: Prediction = 1, Confidence = 60.03%
Frame 15: Prediction = 1, Confidence = 58.97%
Frame 16: Prediction = 1, Confidence = 58.83%
Frame 17: Prediction = 1, Confidence = 58.34%
Frame 18: Prediction = 1, Confidence = 58.34%
Frame 19: Prediction = 1, Confidence = 58.42%
Frame 20: Prediction = 1, Confidence = 59.76%
Frame 21: Prediction = 1, Confidence = 59.42%
Frame 22: Prediction = 1, Confidence = 63.0

In [60]:
'''# Convert to TorchScript
scripted_model = torch.jit.script(model)
scripted_model.save("C:/Users/alapa/Documents/Deepfake_detection_using_deep_learning-master/Deepfake_detection_using_deep_learning-master/model_scripted.pt")
'''

'# Convert to TorchScript\nscripted_model = torch.jit.script(model)\nscripted_model.save("C:/Users/alapa/Documents/Deepfake_detection_using_deep_learning-master/Deepfake_detection_using_deep_learning-master/model_scripted.pt")\n'